In [ ]:
import ast
import json
import os
import sys
import timeit

from collections import Counter


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN

from brtdevkit.core.db.athena import AthenaClient
from brtdevkit.data import Dataset

In [ ]:
europa_path = '/home/alexli/JupiterCVML-master/europa/base/src/europa'
if europa_path not in sys.path:
    sys.path.append(europa_path)

## Set up important constants and info

For example, find the previous training data and make sure it is excluded. We specifically need to look at previous HALO data, but we should also look at previous core train data as well long term.

For now, we don't do this because the query takes a very long time.

In [ ]:
KNOWN_TRAIN_DATASETS = [
    "halo_rgb_stereo_train_v6_2"
]
KNOWN_PRODUCTIVITY_DATASETS = [
    # DAY
    "20230929_halo_rgb_productivity_day_candidate_10_dirty",
    "20230929_halo_rgb_productivity_day_candidate_8_dirty"
    "20230929_halo_rgb_productivity_day_candidate_14_dirty",
    "20230929_halo_rgb_productivity_day_candidate_13_dirty"
    "20230929_halo_rgb_productivity_day_candidate_6_dirty",
    "20230929_halo_rgb_productivity_day_candidate_4_dirty",
    # "20230925_halo_rgb_productivity_day_candidate_3",
    # "20230925_halo_rgb_productivity_day_candidate_2",
    "20230912_halo_rgb_productivity_day_candidate_1",
    "20230912_halo_rgb_productivity_day_candidate_0",
    # NIGHT
    "20230929_halo_rgb_productivity_night_candidate_4_dirty",
    "20230929_halo_rgb_productivity_night_candidate_3_dirty",
    "20230912_halo_rgb_productivity_night_candidate_0",
    # "20230925_halo_rgb_productivity_night_candidate_1",
    # "20230925_halo_rgb_productivity_night_candidate_2",
]
GEOHASH_SHORT = 7

In [ ]:
# get the existing train datasets' short geohashes to exclude
dfs = []
for dataset_name in KNOWN_TRAIN_DATASETS:
    dfs.append(Dataset.retrieve(name=dataset_name).to_dataframe())

In [ ]:
# get the existing productivity datasets' short geohashes to exclude (optional)
for dataset_name in KNOWN_PRODUCTIVITY_DATASETS:
    dfs.append(Dataset.retrieve(name=dataset_name).to_dataframe())

In [ ]:
train_df = pd.concat(dfs)

In [ ]:
train_df["geohash_short"] = train_df["geohash"].apply(lambda x: x[:GEOHASH_SHORT])

In [ ]:
EXCLUDE_GEOHASHES = list(set(train_df["geohash_short"]))
print(len(EXCLUDE_GEOHASHES))

In [ ]:
train_df["robot_name"].value_counts()

## Construct the Athena query for the candidate data

There are a few things that we are looking for in this Athena query.

1. Cannot have overlapping geohashes with the `EXCLUDE_GEOHASHES` string above.
2. Must have a valid geohash. 
3. Must have images from all cams present. This requires a few different checks
   described in the next cell.

### Querying for images from all cams

Querying for sets where all cams are available is somewhat tricky with Halo. There are
a few criteria that need to be satisfied.

1. For a given group ID, if it is a tractor pod it must have 8 images present. If it is
   an implement pod, it must have 4 images present.
2. For each group ID, there must be 

In [ ]:
athena = AthenaClient()

In [ ]:
left_cameras = ('T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I02')
IMPLEMENT_CAMS = ('I01', 'I02', 'I03', 'I04')
FRONT_POD_CAMS = ('T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07', 'T08')
REAR_POD_CAMS = ('T09', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16')

In [ ]:
query = f"""
    SELECT id, geohash, group_id, collected_on, robot_name, bag_name, camera_location,
        operation_time, gps_can_data__json, special_notes, soil_color, operating_field_name,
        implement, state, weather, field_conditions, terrain_type, farm, hard_drive_name
    FROM image_jupiter
    WHERE sensor_type = 'VD6763'
    AND robot_name NOT LIKE 'bedrock%'
    AND gps_can_data__json IS NOT NULL
    AND geohash NOT LIKE '7zzzz%'
    AND SUBSTRING(geohash, 1, {GEOHASH_SHORT}) NOT IN {tuple(EXCLUDE_GEOHASHES)}
    ORDER BY "collected_on"
"""
# AND collected_on BETWEEN TIMESTAMP '2023-08-01 00:00:00' AND TIMESTAMP '2024-08-03 00:00:00'

In [ ]:
start_time = timeit.default_timer()
df = athena.get_df(query)
print(timeit.default_timer() - start_time)

In [ ]:
df["geohash_short"] = df["geohash"].apply(lambda x: x[:GEOHASH_SHORT])

In [ ]:
df['gps_can_data'] = df["gps_can_data__json"].apply(lambda x: json.loads(x))
df['speed'] = df['gps_can_data'].apply(lambda x: x.get('speed', np.nan))

In [ ]:
df = df[df["speed"] > 0.1] # good to actually get moving sequences, but might be too restrictive

In [ ]:
df = df[df["speed"] < 30] # try to remove images where we're just on a road

In [ ]:
len(df)

In [ ]:
# optional visualization of stats
print(df["soil_color"].value_counts(), end="\n\n")
print(df["weather"].value_counts(), end="\n\n")
print(df["field_conditions"].value_counts(), end="\n\n")
print(df["operation_time"].value_counts(), end="\n\n")
print(df["terrain_type"].value_counts(), end="\n\n")
print(df["implement"].value_counts(), end="\n\n")
print(df["state"].value_counts(), end="\n\n")
print(df["state"].isna().value_counts(), end="\n\n")

In [ ]:
# drop all images without full pods
# counter essentially checks whether unique elements of a list are a permutation of each other
def is_full_pod(list1):
    l1_counter = Counter(list1)
    return (
        l1_counter == Counter(IMPLEMENT_CAMS)
        or l1_counter == Counter(FRONT_POD_CAMS)
        or l1_counter == Counter(REAR_POD_CAMS)
    )

df_only_valid = df.groupby("group_id").filter(
    lambda x: is_full_pod(x["camera_location"].tolist())
)

In [ ]:
print(len(df_only_valid))
print(df_only_valid["state"].isna().value_counts())

## Find full sequences

The next step is to find sequences of images with lengths of something like 5 minutes.
These need **completely full** pods, so all three pods need to be full, and each group
of images should be within 1 second of each other from the pods.

(The exact details of this may change - for example loosening the number of seconds between images
or whether the implement pod is required.)

The idea here is to use a graph building algorithm similar to DBSCAN, where the distance
between nodes is based on the `collected_on` timestamp. Images belonging to the same
cluster can be considered part of a sequence. Checks are made to ensure that the correct
machine is used when grouping images together.

In [ ]:
def sequence_graph(df: pd.DataFrame, seconds_apart: float) -> pd.DataFrame:
    """
    Basically runs a DBSCAN algorithm, using the collected on as the metric for which we
    are finding distance, and using an epsilon in seconds.

    Ideally, this should find connected sequences of images. This should be run on dataframes
    for each machine - which is a combination of three robot names (one for each pod).
    """
    # convert collected-on timestamp to to posix float
    df["posix_timestamp"] = df["collected_on"].apply(lambda x: pd.Timestamp(x).timestamp())
    
    # convert the posix timestamp to numpy array
    timestamps = df["posix_timestamp"].to_numpy().reshape((-1, 1))
    try:
        dbscan = DBSCAN(eps=seconds_apart).fit(X=timestamps)
        df["sequence_id"] = dbscan.labels_.astype(int)
    except:
        print("Warning: empty dataframe.")
    return df

In [ ]:
# split the valid dataframe by machine
df_only_valid["robot_name"].value_counts()

In [ ]:
HH0 = ["halohitchhiker_101", "halohitchhiker_102", "halohitchhiker_103"]
HH1 = ["halohitchhiker_111", "halohitchhiker_112", "halohitchhiker_113"]
HH2 = ["halohitchhiker_121", "halohitchhiker_122", "halohitchhiker_123"]
HH6 = ["halohitchhiker_161", "halohitchhiker_162", "halohitchhiker_163"]
HH20 = ["halohitchhiker_201", "halohitchhiker_202", "halohitchhiker_203"]

In [ ]:
df_hh0 = df_only_valid[df_only_valid["robot_name"].isin(HH0)].copy()
df_hh1 = df_only_valid[df_only_valid["robot_name"].isin(HH1)].copy()
df_hh2 = df_only_valid[df_only_valid["robot_name"].isin(HH2)].copy()
# df_hh6 = df_only_valid[df_only_valid["robot_name"].isin(HH6)].copy()
# df_hh20 = df_only_valid[df_only_valid["robot_name"].isin(HH20)].copy()

In [ ]:
df_hh0 = sequence_graph(df_hh0, seconds_apart=20)
df_hh1 = sequence_graph(df_hh1, seconds_apart=20)
df_hh2 = sequence_graph(df_hh2, seconds_apart=20)
# df_hh6 = sequence_graph(df_hh6, seconds_apart=20)
# df_hh20 = sequence_graph(df_hh20, seconds_apart=20)

In [ ]:
print(len(df_hh0["sequence_id"].unique()))
print(len(df_hh1["sequence_id"].unique()))
print(len(df_hh2["sequence_id"].unique()))
# print(len(df_hh6["sequence_id"].unique()))
# print(len(df_hh20["sequence_id"].unique()))

In [ ]:
df_all_bots = pd.concat([df_hh0, df_hh1, df_hh2])
df_all_bots["sequence_id"] = df_all_bots.apply(lambda x: str(int(x["sequence_id"])) + f"_hh{x['robot_name'][-3:-1]}", axis=1)

In [ ]:
# here we can focus on more metadata depending on our needs
# a few examples are provided and commented out
# operation time
# df_all_bots = df_all_bots[df_all_bots["operation_time"] == "nightime"]

# soil color
# df_all_bots = df_all_bots[df_all_bots["soil_color"] == "loamy"]

# field conditions (e.g. crop type)
# df_all_bots = df_all_bots[(df_all_bots["field_conditions"] != "corn") | df_all_bots["field_conditions"].isna()]

# weather conditions
# df_all_bots = df_all_bots[df_all_bots["weather"] == "sunny"]

# terrain type
df_all_bots = df_all_bots[(df_all_bots["terrain_type"] != "headlands") | df_all_bots["terrain_type"].isna()]

# fields
EXCLUDE_FIELDS = [ # generally fields we already have represented
    # "Field 7",
    "0929-Leka Homeplace",
    "1101- Island  S",
    "1102-Schien W",
    "1100- Island  N",
]
df_all_bots["operating_field_name"].fillna("unknown", inplace=True)
df_all_bots = df_all_bots[~(df_all_bots["operating_field_name"].isin(EXCLUDE_FIELDS))]
# df_all_bots = df_all_bots[df_all_bots["operating_field_name"] == "1106- Mitchel"]

# geohash
df_all_bots = df_all_bots[~df_all_bots["geohash"].str.startswith("9xj") & ~df_all_bots["geohash"].str.startswith("dp0")]

# implement
# df_all_bots = df_all_bots[df_all_bots["implement"] == "BR96"]

# state
df_all_bots["state"].fillna("unknown", inplace=True)
df_all_bots = df_all_bots[(~(df_all_bots["state"] == "Illinois") & ~(df_all_bots["state"] == "Colorado"))]
print(df_all_bots["state"].value_counts())

print(len(df_all_bots))

In [ ]:
# filter out bad sequences
# these might be sequences with missing VPUs
# or sequences that are too short
def check_sequence(x) -> bool:
    # check bad vpu
    if len(x["camera_location"].value_counts()) < 16: # allow sequences without implement cams
        return False
    # check too short
    if x["posix_timestamp"].max() - x["posix_timestamp"].min() < 300: # 5 min, can change this
        return False
    # if x["posix_timestamp"].max() - x["posix_timestamp"].min() < 120: # 2 min, can change this
    #     return False
    # check bad speed (too slow or stationary)
    if x["speed"].max() < 1:
        return False
    # check big discepancy between camera location value counts
    # cam_value_counts = x["camera_location"].value_counts()
    # if (cam_value_counts.max() - cam_value_counts.min()) * 1.0 / (cam_value_counts.max()) > 0.1:
    #     return False
    return True

In [ ]:
df_tmp = df_all_bots.groupby("sequence_id").filter(check_sequence)

In [ ]:
pd.set_option("display.max_rows", 200)
df_tmp.groupby(["sequence_id", "state"])["operation_time"].value_counts()

In [ ]:
print(df_tmp["sequence_id"].value_counts())

In [ ]:
# grab only the images of a specific sequence (see output of above cell for sequence names)
df_tmp = df_all_bots[df_all_bots["sequence_id"] == "5_hh10"]

In [ ]:
print(df_tmp["posix_timestamp"].max() - df_tmp["posix_timestamp"].min())
# print(df_tmp["posix_timestamp"].max())

In [ ]:
# this cell just visualizes some stats - not required
print(df_tmp["camera_location"].value_counts())
# print(df_tmp["robot_name"].value_counts())
print(df_tmp["field_conditions"].value_counts())
print(df_tmp["farm"].value_counts())
print(df_tmp["operating_field_name"].value_counts())
print(df_tmp["collected_on"].min())
print(df_tmp["collected_on"].max())
print(df_tmp["speed"].max())
print(df_tmp["state"].value_counts())
print(df_tmp["operation_time"].value_counts())
print(df_tmp["weather"].value_counts())
print(df_tmp[(df_tmp["camera_location"] == "T12")]["id"].tolist())

In [ ]:
# sometimes we only want specific bags of a sequence, since the full sequence may have some issues
# df_tmp = df_tmp[(df_tmp["bag_name"] == "07_20_2023-16_45_47") | (df_tmp["bag_name"] == "07_20_2023-16_45_45")]
# print(len(df_tmp))

In [ ]:
# create dataset candidates
image_ids = list(set(df_tmp["id"].tolist()))
print(len(image_ids))

In [ ]:
Dataset.create(
    name="20230929_halo_rgb_productivity_day_candidate_14_dirty",
    description="A day sequence, roughly 40 minutes long, ~1765 images per camera. Ground has low corn residue, loamy soil, some objects on horizon far away.",
    kind=Dataset.KIND_IMAGE,
    image_ids=image_ids,
)

In [ ]:
Dataset.create(
    name="20230929_halo_rgb_productivity_night_candidate_4_dirty",
    description="A night sequence, roughly 141 minutes long, ~4600 images per camera. Long sequence in what looks like pretty much a dirt field. Starts a bit after sundown (no sun in frame) to night. Some images especially around implement at night have minimal features other than dust.",
    kind=Dataset.KIND_IMAGE,
    image_ids=image_ids,
)